# CSI 4142 - Introduction to Data Science
# Assignment 2: Data cleaning

Shacha Parker (300235525)\
Callum Frodsham and (300199446)\
Group 79

### Setup Instructions To Reproduce this Data Cleaning Notebook:
(Step 1 Optional)
1. Create a virtual python environment in the project directory (if you want) for all of the packages required:  
``` 
python -m venv .venv
```
To enter the virutal environment: 
```
.venv/Scripts/activate.ps1 # on windows
source .venv/bin/activate # on mac/linux
```
2. Download all of the required packages (run in cmd/shell of choice):
```
pip install jupyter
pip install ipykernel
pip install pandas
pip install numpy
```
3. VSCode: Ensure you have the correct python kernel selected!
<br> 
If you are using a virtual environment, make sure to select the python interpreter for that virtual environment otherwise this will not work! If you have everything done globally, then just make sure the correct python kernel you are using is selected.

In [803]:
# Initial imports
import numpy as np
import pandas as pd
import re

<h1>Dataset 1: Netflix Movies and TV Shows</h1>
<h3>Clean Data Checking</h3>

Author: Shivam Bansal
<br>
Purpose: This dataset was made to provide insights on the shows and movies that Netflix is hosting on their platform. For example, these insights could be used to see what type of content the platform is missing, or what type of content they have too much of.
<br>
Shape: This dataset is composed of 12 columns, and 8810 rows.
<br><br>
Link: <a href="https://www.kaggle.com/datasets/shivamb/netflix-shows"> Mobile Device Usage and User Behavior</a>
<br>
<h3>Dataset Feature List: </h3>
<ol>
    <li>Show Id:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The show_id is a unique ID that is assigned to each show/movie. There are 8807 entries ranging from s1 to s8807.
    </li>
    <br>
    <li>Type
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description Informs the viewer whether the content is a tv show or a movie.
        </li>
    <br>
    <li>Title:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The title name of the tv show or movie.
        </li>
    <br>
    <li>Director:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The name of the person(s) who directed the tv show or movie. 
        </li>
    <br>
    <li>Cast:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The name(s) of the notable actor(s) who acted in the tv show or movie.
        </li>
    <br>
    <li>Country:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The countries/country where the tv show or movie was produced.
        </li>
    <br>
    <li>Date Added:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The date the show or movie was added to Netflix.
        </li>
    <br>
    <li>Release Year:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The year in which the tv show or movie was originally released.
        </li>
    <br>
    <li>Rating:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: This rating indicates the acceptable age of viewing for the tv show or movie.
        </li>
    <br>
    <li>Duration:
    <br>
    Feature Type: Mixed Type - Numerical Continuous - Categorical Ordinal 
    <br>
    Description: The duration of the movie in minutes, or if it is a tv show, in seasons.
        </li>
    <br>
    <li>Genre/Listed In:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The genres/subgenres the tv show or movie falls in.
        </li>
    <br>
    <li>Description:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The description of the tv show or movie.
        </li>
</ol>

In [804]:
# load the dataset: 
dataset = pd.read_csv("https://raw.githubusercontent.com/CLFrod/Assignment2CSI4142/refs/heads/master/netflix_titles.csv")

<h4>1. Range Check:</h4>
<p>
In this test, we will verify the range of a numerical value. The range is the minimum and maximum values that an attribute can have.
</p>

In [805]:
# Please enter the various attributes below to perform the tests:
range_attributes = ['release_year', 'date_added']
# attribute selection:
test_range_attribute = 'release_year'
# Minimum:
range_minimum = 1888
# Maximum:
range_maximum = 2025

In [806]:
# Checker Code
range_series = dataset[test_range_attribute].between(range_minimum, range_maximum)

# changing the booleans in range_series from True to False and vice versa
# values that were previously not in range (False) are true!
not_range_series = ~range_series

# check if the range series is empty
empty_range = dataset[not_range_series].empty
# if the series is not empty, 
if not empty_range:
    out_of_range_count = 0
    for val in not_range_series:
        out_of_range_count += 1
    print(out_of_range_count)
# if the series is empty
else:
    print("All values fall within specified paramters!")

All values fall within specified paramters!


<p style ="font-size:20px">Range check Findings: </p>
No range errors were detected based on the provided parameters.
The dataset's 'release_year' feature only has years that fit within logical parameters.

<h4>2. Format Check:</h4>
<p>

</p>

In [807]:
# Please enter the various attributes below to perform the tests:
format_attributes = ['date_added', 'release_year']

test_format_attribute = format_attributes[0]


In [808]:
# Checker Code
# get the df of dates added
date_added_df = dataset['date_added']

#convert to datetime
converted_dates = pd.to_datetime(date_added_df, format="%B %d, %Y", errors="coerce")

# get the incorrectly formatted dates
incorrectly_formatted_dates = converted_dates.isna()
print(dataset[incorrectly_formatted_dates][test_format_attribute])


6066                   NaN
6079        August 4, 2017
6174                   NaN
6177     December 23, 2018
6213     December 15, 2018
               ...        
8539       January 8, 2018
8557     December 25, 2015
8684        August 1, 2017
8712        March 31, 2017
8755      November 1, 2016
Name: date_added, Length: 98, dtype: object


<p style ="font-size:20px">Format check Findings: </p>
There are 98 data points that don't follow the correct format of the dates,
For example,
Row 6068 is missing a date, and thus is technically not following the correct format.<br>
<br>
Row 8759 has " November 1, 2016". Although technically following correct formatting, it has a leading whitespace which should not be there.

<h4>3. Data Type Check:</h4>
<p>

</p>

In [809]:
# Please enter the various attributes below to perform the tests:
attributes = ['release_year']
data_type_test_attribute = attributes[0]

In [810]:
# Checker Code
# small function to check if val is int
def is_integer(val):
    return isinstance(val, int)

data_type_check = dataset[data_type_test_attribute].apply(is_integer).all()

if data_type_check:
    print("All values are integers.")
else:
    print("Not all values are integers.")


All values are integers.


<p style ="font-size:20px">Data Type check Findings: </p>
No data type errors were detected based on the provided parameters.
The dataset's 'release_year' feature only contains integer values.

<h4>4. Consistency Check</h4>
<p>

</p>

In [811]:
# Please enter the various attributes below to perform the tests:
consistency_attributes = ['release_year', "date_added"]


In [812]:
# Checker Code
# deep copy the dataset
temp_df = dataset.copy(True)
# convert date added to python datetime
temp_df[consistency_attributes[1]] = pd.to_datetime(temp_df[consistency_attributes[1]], errors='coerce')
# get the year only
temp_df['year_added'] = temp_df[consistency_attributes[1]].dt.year

# get the dates that aren't consistent
inconsistent_dates = temp_df[temp_df['year_added'] < dataset[consistency_attributes[0]]]

print(inconsistent_dates[consistency_attributes[0]])
print(inconsistent_dates.size)

1551    2021
1696    2021
2920    2021
3168    2020
3287    2020
3369    2020
3433    2020
4844    2019
4845    2019
5394    2018
5658    2018
5677    2017
7063    2019
7112    2016
Name: release_year, dtype: int64
182


<p style ="font-size:20px">Data Type check Findings: </p>
There are 182 data points that are not logically consistent with the date added, and the release year.

For example,

The Film "Hilda" on row 1553 was added in 2020, but the release year says 2021. That should not be possible, unless the series was added to Netflix as a test beforehand or for some other reason.


<h4>5. Uniqueness Check:</h4>
<p>

</p>

In [813]:
# Please enter the various attributes below to perform the tests:
uniqueness_attributes = ['show_id','title']

In [814]:
# Checker Code

if dataset[uniqueness_attributes[0]].is_unique:
    print("All " + uniqueness_attributes[0] + "s are unique values.")
else:
    print("All " + uniqueness_attributes[0] + "s are not unique values.")



All show_ids are unique values.


<p style ="font-size:20px">Uniqueness check Findings: </p>
No uniqueness errors were detected based on the provided parameters!

<h4>6. Presence Check:</h4>
<p>

</p>

In [891]:
# Please enter the various attributes below to perform the tests:
presence_attributes = ['title', 'date_added']

In [910]:
# Checker Code
# get 
presence_series = dataset[presence_attributes[1]].isnull()

print("Rows with missing "+ presence_attributes[1] + ":")
print(dataset[presence_series]['title'])

Rows with missing date_added:
6066    A Young Doctor's Notebook and Other Stories
6174                Anthony Bourdain: Parts Unknown
6795                                        Frasier
6806                                        Friends
6901                                Gunslinger Girl
7196                                       Kikoriki
7254                            La Familia P. Luche
7406                                          Maron
7847                                   Red vs. Blue
8182                   The Adventures of Figaro Pho
Name: title, dtype: object


<p style ="font-size:20px">Presence check Findings: </p>
There are 10 data points in the dataset where the date_added, is empty or null. See for example the following rows:

show_id / title / date_added / release_year
s6067 / A Young Doctor's Notebook and Other Stories / / 2013


<h4>7. Length Check:</h4>
<p>

</p>

In [911]:
# Please enter the various attributes below to perform the tests:
attributes = []

In [912]:
# Checker Code

findings report

<h4>8. Look-up Check:</h4>
<p>

</p>

In [913]:
# Please enter the various attributes below to perform the tests:
attributes = []

In [914]:
# Checker Code

findings report

<h4>9. Exact Duplicate Check</h4>
<p>

</p>

In [915]:
# Please enter the various attributes below to perform the tests:
attributes = []

In [916]:
# Checker Code

findings report

<h4>10. Near Duplicate Check:</h4>
<p>

</p>

In [917]:
# Please enter the various attributes below to perform the tests:
attributes = []


In [918]:
# Checker Code

findings report

<h3>References:</h3>
<ul>
<li>
<a href="https://www.w3schools.com/python/python_datetime.asp"> Python Date time formatting</a>
</li>
<li>
<a href="https://stackoverflow.com/questions/402504/how-to-determine-a-python-variables-type"> Check Variable Type</a>
</li>
</ul>